Dowload the Required Modules

In [ ]:
pip install requests beautifulsoup4

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
pip install openpyxl

Open the Hyperlink

In [2]:
import openpyxl

workbook = openpyxl.load_workbook('/content/drive/MyDrive/20211030 Test Assignment/Input.xlsx')
sheet = workbook.active

hyperlinks = []

for row in sheet.iter_rows():
    for cell in row:
        if cell.hyperlink:
            hyperlinks.append(cell.hyperlink.target)


# Extracting Titles

In [3]:
import requests
from bs4 import BeautifulSoup

article_titles = []

for link in hyperlinks:
    response = requests.get(link)
    soup = BeautifulSoup(response.content, 'html.parser')
    title = soup.find('h1').text
    article_titles.append(title)

# URL_ID save it in a text file

with open('URL_ID.txt', 'w') as file:
    for title in article_titles:
        file.write(title + '\n')


# Writing Output in the Excel File

In [4]:
headTexts = []
with open('URL_ID.txt', 'r') as file:
    headTexts = file.readlines()
    file.read()

# Using NLP techniques to perform textual analysis

In [ ]:
import nltk
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')
nltk.download('averaged_perceptron_tagger')

Removing Stopwords

In [6]:
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import sent_tokenize
from nltk.stem import WordNetLemmatizer


stopwords = set(nltk.corpus.stopwords.words('english'))
lemmatizer = WordNetLemmatizer()
title_list = []

def remove_stopwords(text):
    headTexts = text
    words = []
    for sentence in headTexts:
        tokens = nltk.word_tokenize(sentence)
        tokens = [lemmatizer.lemmatize(word) for word in tokens if word.isalnum()]
        tokens = [word for word in tokens if word not in stopwords]
        words.append(tokens)

    return words

title_list = remove_stopwords(headTexts)


In [ ]:
import nltk
nltk.download('opinion_lexicon')
nltk.download('punkt')


In [ ]:
import nltk
from nltk.sentiment.vader import SentimentIntensityAnalyzer

nltk.download('vader_lexicon')

sid = SentimentIntensityAnalyzer()

joined_sentences = [' '.join(sentence) for sentence in title_list]
polarity_scores = []

for sentence in joined_sentences:
    scores = sid.polarity_scores(sentence)
    compound = scores['compound']
    if compound >= 0.05:
        sentiment = "Positive"
    elif compound <= -0.05:
        sentiment = "Negative"
        # print(f"Sentence: {sentence}\nSentiment: {sentiment}, Polarity Score: {scores}, Sentence Length: {len(sentence)}\n")
    polarity_scores.append(scores)


In [9]:
# Extract the 'polarity' value
polarity_values = [score['compound'] for score in polarity_scores[:-1]]
pos_values = [score['pos'] for score in polarity_scores[:-1]]
neg_values = [score['neg'] for score in polarity_scores[:-1]]

In [10]:
from openpyxl import load_workbook

workbook = load_workbook('/content/drive/MyDrive/20211030 Test Assignment/Output Data Structure.xlsx')
sheet = workbook.active

header = [cell.value for cell in sheet[1]]
polarity_col_index = header.index('POSITIVE SCORE') + 1

for i, score in enumerate(pos_values, start=2):
    sheet.cell(row=i, column=polarity_col_index, value=score)

# Save the workbook
workbook.save('Output Data Structure.xlsx')

In [11]:
from openpyxl import load_workbook

workbook = load_workbook('Output Data Structure.xlsx')
sheet = workbook.active

header = [cell.value for cell in sheet[1]]
polarity_col_index = header.index('NEGATIVE SCORE') + 1

for i, score in enumerate(neg_values, start=2):
    sheet.cell(row=i, column=polarity_col_index, value=score)

workbook.save('Output Data Structure.xlsx')

In [12]:
from openpyxl import load_workbook

workbook = load_workbook('Output Data Structure.xlsx')
sheet = workbook.active

header = [cell.value for cell in sheet[1]]
polarity_col_index = header.index('POLARITY SCORE') + 1

for i, score in enumerate(polarity_values, start=2):
    sheet.cell(row=i, column=polarity_col_index, value=score)

workbook.save('Output Data Structure.xlsx')

In [13]:
insertion_values = ['POSITIVE SCORE','NEGATIVE SCORE','POLARITY SCORE','SUBJECTIVITY SCORE','AVG SENTENCE LENGTH','PERCENTAGE OF COMPLEX WORDS','FOG INDEX','AVG NUMBER OF WORDS PER SENTENCE','COMPLEX WORD COUNT','WORD COUNT','SYLLABLE PER WORD','PERSONAL PRONOUNS','AVG WORD LENGTH']

In [14]:
from openpyxl import load_workbook

avg_senteces_length = []
for word in title_list:
    avg_senteces_length.append(len(word)/len(title_list))

workbook = load_workbook('Output Data Structure.xlsx')
sheet = workbook.active

header = [cell.value for cell in sheet[1]]
polarity_col_index = header.index('AVG SENTENCE LENGTH') + 1

for i, score in enumerate(avg_senteces_length, start=2):
    sheet.cell(row=i, column=polarity_col_index, value=score)

workbook.save('Output Data Structure.xlsx')

In [15]:
# Fog Index = 0.4 * (Average Sentence Length + Percentage of Complex words)
avg_senteces_length = []
complex_words = []

for word in title_list:
    avg_senteces_length.append(len(word)/len(title_list))
for title in title_list:
  for word in title:
    if word.islower():
      complex_words.append(word)


In [ ]:
import nltk
nltk.download('punkt')
nltk.download('cmudict')


In [ ]:

import nltk
from nltk.corpus import cmudict

d = cmudict.dict()

def count_syllables(word):
    if word.lower() in d:
        return [len(list(y for y in x if y[-1].isdigit())) for x in d[word.lower()]][0]
    else:
        return 0

def is_complex_word(word, syllable_threshold=3, length_threshold=7):
    syllables = count_syllables(word)
    return syllables >= syllable_threshold or len(word) >= length_threshold

def find_complex_words(sentence, syllable_threshold=3, length_threshold=7):
    words = nltk.word_tokenize(sentence)
    complex_words = [word for word in words if is_complex_word(word, syllable_threshold, length_threshold)]
    return complex_words

complex_words = []
for i in range(len(joined_sentences)):
  sentence = joined_sentences[i]
  complex_words.append(find_complex_words(sentence))

complex_words

In [ ]:
len(pos_values), len(neg_values), len(joined_sentences)

In [49]:
list1 = []
for i in range(len(joined_sentences)-1):
  list1.append(float(((pos_values[i] + neg_values[i])/len(joined_sentences[i]) + 0.000001)))

subj_score = []
for i in range(len(list1)):
  subj_score.append(f"{list1[i]:.10f}")

In [52]:
workbook = load_workbook('Output Data Structure.xlsx')
sheet = workbook.active

header = [cell.value for cell in sheet[1]]
polarity_col_index = header.index('SUBJECTIVITY SCORE') + 1

for i, score in enumerate(subj_score, start=2):
    sheet.cell(row=i, column=polarity_col_index, value=score)

workbook.save('Output Data Structure.xlsx')

In [59]:
sheet = workbook['Sheet1']
sheet.delete_cols(8, 1)
sheet.delete_cols(11, 1)
sheet.delete_cols(12, 1)

workbook.save('Output Data Structure.xlsx')
